# Example 2: class Beam from wave-propagation codes output

**Goal.** Build standard beams from SRW intensity maps in two ways and compare:
1) **Far-field only (FF)** → assumes a point source at `z0` (positions set by model).
2) **Near-field + Far-field (NF+FF)** → samples both position *(X,Y)* and angle *(dX,dY)*.

In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'CECILL-2.1'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '2025.11.10'
__changed__ = '2025.11.10'

import h5py
import matplotlib.pyplot as plt

import barc4beams as b4b

## 0) User parameters

In [ ]:
N_RAYS    = 50000
THRESHOLD = 1E-4
JITTER    = True
SEED      = 42
Z0        = 0.0

## 1) Load SRW intensity maps (HDF5)
We need:
- **far_field** dictionary with `intensity`; `x_axis`  and `y_axis` in (rad)
- **near_field** dictionary with `intensity`; `x_axis` and  `y_axis` in (m)
- the attribute `energy`

In [ ]:
path = "./beams/srw_undulator_emission_intensity_distribution.h5"

with h5py.File(path, "r") as f:
    energy = float(f["energy_eV"][()])
    dict_ff = {k: f["far_field"][k][()] for k in ("intensity", "x_axis", "y_axis")}
    dict_nf = {k: f["near_field"][k][()] for k in ("intensity", "x_axis", "y_axis")}

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4), constrained_layout=True)

im0 = axes[0].pcolormesh(
    dict_nf["x_axis"] * 1e6,   
    dict_nf["y_axis"] * 1e6,
    dict_nf["intensity"],
    shading="auto",
    cmap="turbo"
)
axes[0].set_title("Spatial distribution (near field)")
axes[0].set_xlabel("x [µm]")
axes[0].set_ylabel("y [µm]")
axes[0].set_aspect("equal")
fig.colorbar(im0, ax=axes[0], label="Intensity [a.u.]")

im1 = axes[1].pcolormesh(
    dict_ff["x_axis"] * 1e6,   
    dict_ff["y_axis"] * 1e6,
    dict_ff["intensity"],
    shading="auto",
    cmap="turbo"
)
axes[1].set_title("Angular distribution (far field)")
axes[1].set_xlabel("x' [µrad]")
axes[1].set_ylabel("y' [µrad]")
axes[1].set_aspect("equal")
fig.colorbar(im1, ax=axes[1], label="Intensity [a.u.]")

plt.show()

## 2) Generate beams:
- from **angular distribution-only (FF)**: positions are set to (0, 0) at `z0`; only angles are sampled from far-field.
- from **spatial and angular distributions(NF+FF)**: positions from near-field and angles from far-field (with independent RNG streams).

> **Note on sampling algorithm**  
>
> The sampling algorithm used in `Beam.from_intensity` is
> based on the approach described in **Section 2.1** of  
> Rebuffi et al. **J. Synchrotron Rad.** 27, 1108–1120 (2020)

### angular distribution-only (FF)

In [ ]:
beam_from_ff = b4b.Beam.from_intensity(
    far_field=dict_ff,
    near_field=None,
    n_rays=N_RAYS,
    energy=energy,
    jitter=JITTER,
    threshold=THRESHOLD,
    seed=SEED,
    z0=Z0,
    polarization_degree=1.0,
)

In [ ]:
stats = beam_from_ff.stats(verbose=True)

In [ ]:
plot = beam_from_ff.plot_beam(mode="hist2d", aspect_ratio=True, color=3, path=None, bins=100)
plot = beam_from_ff.plot_divergence(mode="hist2d", aspect_ratio=True, color=3, path=None, bins=100)

In [ ]:
plot = beam_from_ff.plot_caustic(which="both", n_points=101, xy_range=[-5, 5], start=-0.1, finish=0.1, color=3)

### spatial and angular distributions(NF+FF)

In [ ]:
beam_from_ff_and_nf = b4b.Beam.from_intensity(
    far_field=dict_ff,
    near_field=dict_nf,
    n_rays=N_RAYS,
    energy=energy,
    jitter=JITTER,
    threshold=THRESHOLD,
    seed=SEED,
    z0=Z0,
    polarization_degree=1.0,
)

In [ ]:
stats = beam_from_ff_and_nf.stats(verbose=True)

In [ ]:
plot = beam_from_ff_and_nf.plot_beam(mode="hist2d", aspect_ratio=True, color=3, path=None, bins=100)
plot = beam_from_ff_and_nf.plot_divergence(mode="hist2d", aspect_ratio=True, color=3, path=None, bins=100)

In [ ]:
plot = beam_from_ff_and_nf.plot_caustic(which="both", n_points=101, xy_range=[-5, 5], start=-0.1, finish=0.1, color=3)

## 3) Notes & expected differences

- **FF-only** corresponds to a *point source* located at `z0`. 
- **NF+FF** uses both maps, thus reproducing the beam’s **spot size** and **angular distribution**.
- Use `THRESHOLD` to suppress numerical noise in low-signal regions (applied independently to NF and FF).
- `JITTER=True` adds sub-pixel randomness so the samples don’t lie exactly on grid centers.

> You can reuse all plotting calls from Example 1 (footprint, divergence, energy, etc.) on either beam.